In [ ]:
# # --------------------------------------------------
# # 1️⃣ Mount Google Drive (optional, for cache)
# # --------------------------------------------------
# from google.colab import drive
# import os

# if not os.path.ismount("/content/drive"):
#     drive.mount("/content/drive")
# else:
#     print("📦 Google Drive already mounted")

# --------------------------------------------------
# 2️⃣ Clone fenicsx-colab repository (idempotent)
# --------------------------------------------------
from pathlib import Path
import subprocess

REPO_URL = "https://github.com/seoultechpse/fenicsx-colab.git"
ROOT = Path("/content")
REPO_DIR = ROOT / "fenicsx-colab"

def run(cmd):
    #print("$", " ".join(map(str, cmd)))
    subprocess.run(cmd, check=True)

if not REPO_DIR.exists():
    print("📥 Cloning fenicsx-colab...")
    run(["git", "clone", REPO_URL, str(REPO_DIR)])
elif not (REPO_DIR / ".git").exists():
    raise RuntimeError("Directory exists but is not a git repository")
else:
    print("📦 Repository already exists — skipping clone")

# --------------------------------------------------
# 3️⃣ Run setup_fenicsx.py IN THIS KERNEL (CRITICAL)
# --------------------------------------------------
print("🚀 Running setup_fenicsx.py in current kernel")

USE_CLEAN = False  # <--- Set True to remove existing environment
opts = "--clean" if USE_CLEAN else ""

get_ipython().run_line_magic(
    "run", f"{REPO_DIR / 'setup_fenicsx.py'} {opts}"
)

# ==================================================
# 4️⃣ Sanity check
# ==================================================
try:
    get_ipython().run_cell_magic('fenicsx', '--info -np 4', '')
except Exception as e:
    print("⚠️ %%fenicsx magic not found:", e)

In [ ]:
%%fenicsx -np 4 --time
from pathlib import Path
from mpi4py import MPI
import numpy as np
import pyvista
import dolfinx.plot as plot
from dolfinx.fem import Function, functionspace
from dolfinx.mesh import CellType, create_unit_square

# -----------------------------------------------------------
# 1. 가상 디스플레이 시작 및 오프스크린 설정
# -----------------------------------------------------------
pyvista.start_xvfb()
pyvista.OFF_SCREEN = True  # 이것을 True로 해야 파일 저장이 활성화

# 폴더 생성
out_folder = Path("out_pyvista")
out_folder.mkdir(parents=True, exist_ok=True)

def plot_scalar():
    comm = MPI.COMM_WORLD
    rank = comm.rank

    # Mesh 생성 (쿼드라터럴)
    msh = create_unit_square(
        comm, 12, 12, cell_type=CellType.quadrilateral, dtype=np.float64
    )
    V = functionspace(msh, ("Lagrange", 1))
    u = Function(V, dtype=np.float64)
    u.interpolate(lambda x: np.sin(np.pi * x[0]) * np.sin(2 * x[1] * np.pi))

    # PyVista Grid 변환
    cells, types, x = plot.vtk_mesh(V)
    grid = pyvista.UnstructuredGrid(cells, types, x)
    grid.point_data["u"] = u.x.array

    grid.set_active_scalars("u")
    warped = grid.warp_by_scalar()

    # -----------------------------------------------------------
    # 2. Rank 0인 프로세스만 Plotter를 생성하고 저장 수행
    # -----------------------------------------------------------
    if rank == 0:
        subplotter = pyvista.Plotter(shape=(1, 2))

        # 왼쪽: Scalar Field
        subplotter.subplot(0, 0)
        subplotter.add_text("Scalar contour field", font_size=14, color="black", position="upper_edge")
        subplotter.add_mesh(grid, show_edges=True, show_scalar_bar=True)
        subplotter.view_xy()

        # 오른쪽: Warped Function
        subplotter.subplot(0, 1)
        subplotter.add_text("Warped function", position="upper_edge", font_size=14, color="black")

        sargs = dict(
            height=0.8, width=0.1, vertical=True,
            position_x=0.05, position_y=0.05, fmt="%1.2e",
            title_font_size=40, color="black", label_font_size=25,
        )
        subplotter.set_position([-3, 2.6, 0.3])
        subplotter.set_focus([3, -1, -0.15])
        subplotter.set_viewup([0, 0, 1])
        subplotter.add_mesh(warped, show_edges=True, scalar_bar_args=sargs)

        # -----------------------------------------------------------
        # 3. 조건문 없이 무조건 스크린샷 저장
        # -----------------------------------------------------------
        filename = out_folder / "2D_function_warp.png"
        subplotter.screenshot(
            filename,
            transparent_background=False,
            window_size=[800, 800],
        )
        print(f"이미지가 저장되었습니다: {filename}")

plot_scalar()